# Gravity: Laguna del Maule Bouguer Gravity

This notebook illustrates the SimPEG code used to invert Bouguer gravity data collected at Laguna del Maule volcanic field, Chile. Refer to [Miller et al 2017 EPSL](https://doi.org/10.1016/j.epsl.2016.11.007) for full details.

Originally implemented in the [SimPEG Examples](http://docs.simpeg.xyz/content/examples/04-grav/plot_laguna_del_maule_inversion.html#sphx-glr-content-examples-04-grav-plot-laguna-del-maule-inversion-py)

In [1]:
import deepdish as dd
import os
import tarfile
import SimPEG.PF as PF
from SimPEG import (
    Maps, Regularization, Optimization, DataMisfit,
    InvProblem, Directives, Inversion, Utils
)
from SimPEG.Utils.io_utils import download
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Download the data
url = "https://storage.googleapis.com/simpeg/Chile_GRAV_4_Miller/Chile_GRAV_4_Miller.tar.gz"
downloads = download(url, overwrite=True)
basePath = downloads.split(".")[0]

# unzip the tarfile
tar = tarfile.open(downloads, "r")
tar.extractall()
tar.close()

input_file = os.path.sep.join([basePath, 'LdM_input_file.inp'])

overwriting /Users/lindseyjh/git/simpeg-research/uda-2019-inversion/Chile_GRAV_4_Miller.tar.gz
   saved to: /Users/lindseyjh/git/simpeg-research/uda-2019-inversion/Chile_GRAV_4_Miller.tar.gz
Download completed!


## Input parameters

In [3]:
# Plotting parameters, max and min densities in g/cc
vmin = -0.6
vmax = 0.6

In [4]:
# weight exponent for default weighting
wgtexp = 3.
# %%
# Read in the input file which included all parameters at once
# (mesh, topo, model, survey, inv param, etc.)
driver = PF.GravityDriver.GravityDriver_Inv(input_file)

## forward simulation

In [5]:
# Access the mesh and survey information
mesh = driver.mesh
survey = driver.survey

# define gravity survey locations
rxLoc = survey.srcField.rxList[0].locs

# define gravity data and errors
d = survey.dobs
wd = survey.std

In [6]:
# Get the active cells
active = driver.activeCells
nC = len(active)  # Number of active cells

# Create active map to go from reduce set to full
activeMap = Maps.InjectActiveCells(mesh, active, -100)

# Create static map
static = driver.staticCells
dynamic = driver.dynamicCells

staticCells = Maps.InjectActiveCells(
    None, dynamic, driver.m0[static], nC=nC
)
mstart = driver.m0[dynamic]

# Get index of the center
midx = int(mesh.nCx/2)
# %%
# Now that we have a model and a survey we can build the linear system ...
# Create the forward model operator
prob = PF.Gravity.GravityIntegral(mesh, rhoMap=staticCells,
                                  actInd=active)
prob.solverOpts['accuracyTol'] = 1e-4

# Pair the survey and problem
survey.pair(prob)

## Set up the inversion

In [7]:
# create a custom directive to save the inversion model at every iteration to hdf5

class SaveModelEveryIterationHDF5(Directives.SaveEveryIteration):
    """SaveModelEveryIteration

    This directive saves the model as a numpy array at each iteration. The
    default direcroty is the current directoy and the models are saved as
    `InversionModel-YYYY-MM-DD-HH-MM-iter.h5`
    """
    
    mapping = None
    
    def initialize(self):
        print(
            "SimPEG.SaveModelEveryIteration will save your models as: "
            "'{0!s}###-{1!s}.h5'".format(
                self.directory + os.path.sep, self.fileName
            )
        )

    def endIter(self):
        mesh = self.invProb.dmisfit.prob.mesh
        rhoMap = self.invProb.dmisfit.prob.rhoMap
        model = self.opt.xc
        if self.mapping is not None:
            model = self.mapping * model
        model = model.reshape(mesh.vnC, order="F")
        data = {
            "x": mesh.vectorCCx,
            "y": mesh.vectorCCy, 
            "z": mesh.vectorCCz, 
            "model": model
        }
        dd.io.save(
            '{0!s}{1:03d}-{2!s}.h5'.format(
                self.directory + os.path.sep, self.opt.iter, self.fileName
            ), data
        )

In [8]:
# Apply depth weighting
wr = PF.Magnetics.get_dist_wgt(mesh, rxLoc, active, wgtexp,
                               np.min(mesh.hx)/4.)
wr = wr**2.

# %% Create inversion objects
reg = Regularization.Sparse(
    mesh, indActive=active, mapping=staticCells, gradientType='total'
)
reg.mref = driver.mref[dynamic]
reg.cell_weights = wr * mesh.vol[active]
reg.norms = np.c_[0., 1., 1., 1.]
# reg.norms = driver.lpnorms

# Specify how the optimization will proceed
opt = Optimization.ProjectedGNCG(
    maxIter=20, lower=driver.bounds[0],
    upper=driver.bounds[1], maxIterLS=10,
    maxIterCG=20, tolCG=1e-3
)

# Define misfit function (obs-calc)
dmis = DataMisfit.l2_DataMisfit(survey)
dmis.W = 1./wd

Begin calculation of distance weighting for R= 3.0
Done 0.0 %
Done 10.0 %
Done 20.0 %
Done 30.0 %
Done 40.0 %
Done 50.0 %
Done 60.0 %
Done 70.0 %
Done 80.0 %
Done 90.0 %
Done 100% ...distance weighting completed!!

SimPEG.DataMisfit.l2_DataMisfit assigning default eps of 1e-5 * ||dobs||
Begin linear forward calculation: z
Done 0.0 %
Done 10.0 %
Done 20.0 %
Done 30.0 %
Done 40.0 %
Done 50.0 %
Done 60.0 %
Done 70.0 %
Done 80.0 %
Done 90.0 %
Linear forward calculation ended in: 13.51827597618103 sec


In [9]:
# create the default L2 inverse problem from the above objects
invProb = InvProblem.BaseInvProblem(dmis, reg, opt)

# Specify how the initial beta is found
betaest = Directives.BetaEstimate_ByEig(beta0_ratio=1e-2)

# save the results

save_model_mapping = Maps.InjectActiveCells(mesh, active, np.nan)
save_models = SaveModelEveryIterationHDF5(directory="inversion_results", mapping=save_model_mapping)

# IRLS sets up the Lp inversion problem
# Set the eps parameter parameter in Line 11 of the
# input file based on the distribution of model (DEFAULT = 95th %ile)
IRLS = Directives.Update_IRLS(
    f_min_change=1e-4, maxIRLSiter=40, beta_tol=5e-1,
    betaSearch=False
)

# Preconditioning refreshing for each IRLS iteration
update_Jacobi = Directives.UpdatePreconditioner()

# Create combined the L2 and Lp problem
inv = Inversion.BaseInversion(
    invProb, directiveList=[IRLS, update_Jacobi, betaest, save_models]
)

In [10]:
# Run L2 and Lp inversion
mrec = inv.run(mstart)


    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
Approximated diag(JtJ) with linear operator
SimPEG.SaveModelEveryIteration will save your models as: 'inversion_results/###-InversionModel-2019-11-13-09-37.h5'
model has any nan: 0
=============================== Projected GNCG ===============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
x0 has any nan: 0
   0  2.32e-03  1.30e+06  1.87e+01  1.30e+06    1.95e+02      0              
   1  1.16e-03  3.71e+03  7.00e+06  1.18e+04    1.04e+02      0              
   2  5.81e-04  1.85e+03  8.10e+06  6.56e+03    8.71e+01      0   Skip BFGS  
   3  2.91e-04  8.71e+02  9.27e+06  3.56e+03    7.20e+01      0   Skip BFGS  
   4  1.45e-04  3.74e+02  1.04e+07  1.89e+03    5.69e+01      0   Skip BFGS  
   5  7.26e-05  1.48e+02  1.15e+07  9.

## plot the results

In [11]:
fig, ax = plt.subplots, 
# Plot observed data
Utils.PlotUtils.plot2Ddata(rxLoc, d)

# %%
# Write output model and data files and print misfit stats.

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
iteration = 20
iz = 20

data = dd.io.load(
    f"{save_models.directory}{os.path.sep}{iteration:03d}-{save_models.fileName}.h5"
)

plt.pcolormesh(data["x"], data["y"], data["model"].reshape(mesh.vnC, order="F")[:, :, iz])
plt.title(f"z = {data['z'][iz]}m")

In [ ]:
iy = 29

plt.pcolormesh(data["x"], data["z"], data["model"][:, iy, :].T)
plt.title(f"y = {data['y'][iy]}m")